In [7]:
import numpy as np
import tensorflow as tf
import os
import scipy.misc
from easydict import EasyDict as edict
from models.IDN_YCbCr import *
from utils import *
from ops import *
from dataloader import *

%reload_ext autoreload
%autoreload 2

config = edict()
# training parameters
config.batch_size = 64
config.patch_size = 17 # patch size in LR domain (in HR domain it should be multiplied by scale)
config.mode = "YCbCr" #YCbCr
config.channels = 1
#config.mode = "RGB" #YCbCr
#config.channels = 3
config.scale = 3
config.content_layer = 'relu5_4'
config.learning_rate = 1e-4
config.augmentation = True #data augmentation (flip, rotation)

config.epochs = 500
config.test_every = 1
config.repeat = 100

config.num_d_blocks = 4

# weights for loss 
config.w_losses = {'L1':1, 'MSE':0, 'texture': 0, 'content': 0, 'tv':0}

config.model_name = "IDN-YbCr"

# directories
config.dataset_name = "DIV2K"
config.testset_name = "Set5"#"Set5"
config.train_path_LR = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/train/","%s_LR/*.png" %str(config.dataset_name))
config.train_path_HR = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/train/",str(config.dataset_name),"*.png")
config.valid_path_HR = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/vaild/",str(config.dataset_name),"*.png")
#config.test_path = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/",str(config.testset_name),"*.png")

config.test_path_HR = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/%s/HR/*.png" %str(config.testset_name))
config.test_path_LR = os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/%s/LR/*.png" %str(config.testset_name))
config.vgg_dir = "../vgg_pretrained/imagenet-vgg-verydeep-19.mat"

config.result_dir = os.path.join("./result", config.model_name, "D%d" %config.num_d_blocks)
config.result_img_dir = os.path.join(config.result_dir, "images", config.testset_name)
config.checkpoint_dir = os.path.join(config.result_dir, "model")

if not os.path.exists(config.checkpoint_dir):
    print("creating dir...", config.checkpoint_dir)
    os.makedirs(config.checkpoint_dir)
if not os.path.exists(config.result_dir):
    print("creating dir...", config.result_dir)
    os.makedirs(config.result_dir)
if not os.path.exists(config.result_img_dir):
    print("creating dir...", config.result_img_dir)
    os.makedirs(config.result_img_dir)

In [ ]:
#TODO list
1. data loader 확인 -> OK
2. GAN 빼고 PSNR로만 training -> 되는 것 같은데 PSNR이 낮음
3. discriminator pretrain해서 잘 되는지 확인
4. 전체 training

In [10]:
# load dataset
dataset_LR, dataset_HR = load_dataset(config)

Dataset: DIV2K, 900 images
loading 900 LR images...
loading 900 HR images...
900 image pairs loaded! setting took: 99.0924s


In [80]:
LR_batch, HR_batch, bicubic_batch = get_batch_Y(dataset_LR, dataset_HR, 200, config)
print(LR_batch[0].shape)
x = 197
imageio.imwrite("LR.png", postprocess_Y(LR_batch[x]))
imageio.imwrite("HR.png", postprocess_Y(HR_batch[x]))
imageio.imwrite("bicubic.png", postprocess_Y(bicubic_batch[x]))

(17, 17, 1)


In [8]:
# build model
tf.reset_default_graph()
# uncomment this when only trying to test the model
dataset_LR = []
dataset_HR = []
sess = tf.Session()
model = IDN_YCbCr(sess, config, dataset_LR, dataset_HR)

Creating IDN YCbCrx3
Completed building generator. Number of variables: 62
creating L1 loss, weight:  1
Content loss not available in YCbCr space
no texture loss
no tv loss
no color (MSE) loss


In [16]:
# pretrain discriminator with (phone, dslr) pairs
model.pretrain_discriminator(load = False)

 Discriminator training starts from beginning
------Iteration 0, runtime: 0.241 s, discriminator loss: 1.386617
Discriminator test accuracy: bicubic: 0/200, HR: 200/200
------Iteration 1000, runtime: 103.803 s, discriminator loss: 0.871274
Discriminator test accuracy: bicubic: 200/200, HR: 141/200
------Iteration 2000, runtime: 207.872 s, discriminator loss: 0.518489
Discriminator test accuracy: bicubic: 179/200, HR: 182/200
------Iteration 3000, runtime: 311.988 s, discriminator loss: 0.463568
Discriminator test accuracy: bicubic: 199/200, HR: 160/200
------Iteration 4000, runtime: 416.192 s, discriminator loss: 0.807680
Discriminator test accuracy: bicubic: 180/200, HR: 176/200
pretraining complete


In [18]:
# test discriminator performance for (phone, dslr) pair
model.test_discriminator(200, load = True, model_name = "LPSR-YCbCr-discriminator-pretrained")

Discriminator-pretrained
Loading checkpoints from  checkpoint/DIV2K
INFO:tensorflow:Restoring parameters from checkpoint/DIV2K/Discriminator-pretrained
 [*] Load SUCCESS
Discriminator test accuracy: bicubic: 176/200, HR: 97/200


In [72]:
# train generator & discriminator together
model.train(load = True)

INFO:tensorflow:Restoring parameters from ./result/IDN-YbCr/D4/model/model_best
Continuing from epoch 12
Number of images: 900, batch size: 64, num_repeat: 100 --> each epoch consists of 1406 iterations
------Epoch 13, runtime: 90.787 s, generator loss: 3.524018
image size:  (510, 510, 1) , inference time: 0.040386199951171875
image size:  (288, 288, 1) , inference time: 0.004417896270751953
image size:  (255, 255, 1) , inference time: 0.003637552261352539
image size:  (279, 279, 1) , inference time: 0.00420379638671875
image size:  (342, 228, 1) , inference time: 0.004185676574707031
Test PSNR: 34.016 (best: 34.046 at epoch 9), bicubic: 30.403


KeyboardInterrupt: 

In [9]:
# test trained model
model.test_generator(load = True)

INFO:tensorflow:Restoring parameters from ./result/IDN-YbCr/D4/model/model_best
Continuing from epoch 13


(37.440325387601405, 34.91064951925091)

In [ ]:
# save trained model
model.save()

In [23]:
model.load()

Loading checkpoints from  ./result/IDN-YbCr/D4/model/model_latest
INFO:tensorflow:Restoring parameters from ./result/IDN-YbCr/D4/model/model_latest
Continuing from epoch 1
3.227469512460727685e+01


In [6]:
import skimage.color as sc
import cv2
mean_RGB = np.zeros([1,1,3], dtype = "uint8")
mean_RGB[:,:,:] = [124 ,  117,  104] #np.array([123.68 ,  116.779,  103.939])
mean_ycbcr = sc.rgb2ycbcr(mean_RGB)
print(mean_ycbcr)
mean_RGB2 = np.zeros([1,1,3], dtype = "float32")
mean_RGB2[:,:,:] = [123.68 ,  116.779,  103.939]
mean_ycbcr2 = cv2.cvtColor(mean_RGB2, cv2.COLOR_RGB2YCrCb)
print(mean_ycbcr2)

[[[117.00709412 121.25263137 132.00306667]]]
[[[117.37863    4.992876  -7.079951]]]
